In [1]:
# Create database 
import cv2 as cv
import os
DATA_DIR = './data'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

number_of_classes = 2
dataset_size = 100
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("cannot open camera")
    exit()


def create_data(name):
    class_dir = os.path.join(DATA_DIR,str(name))
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)
    print(f"Collecting data for  {name}")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error, failed to capture frame")
            break

        cv.putText(frame,   "Ready? press Q",   (100,50),   cv.FONT_HERSHEY_SIMPLEX,  1.3,  (0,255,0),   3,   cv.LINE_AA)
        cv.imshow('frame',frame)
        if cv.waitKey(25)==ord('q'):
            break

    counter = 0
    while counter<dataset_size:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame.")
            break
        cv.imshow('frame', frame)
        cv.waitKey(100)
        cv.imwrite(os.path.join(class_dir, '{}.jpg'.format(counter)), frame)
        counter += 1
    


create_data("Adivya")
cap.release()
cv.destroyAllWindows()

In [ ]:
# Generate embeddings
import os
import matplotlib.pyplot as plt
import cv2 as cv
from mtcnn.mtcnn import MTCNN
import numpy as np
from keras_facenet import FaceNet
import imgaug.augmenters as iaa



class faceloading:
    def __init__(self,directory):
        self.directory = directory
        self.target_size = (160,160)
        self.X = []
        self.Y = []
        self.detector=  MTCNN()
        self.augmenter = iaa.Sequential([
            # iaa.Fliplr(0.5),  # 50% chance horizontal flip
            # iaa.GaussianBlur(sigma=(0, 1.0)),
            # iaa.AdditiveGaussianNoise(scale=(10, 20)),
            # iaa.Multiply((0.8, 1.2)),  # Brightness
            # iaa.LinearContrast((0.75, 1.5)),
            iaa.Affine(rotate=(-15, 15)), # Rotate ±15 degrees

            iaa.Sometimes(0.3, iaa.Multiply((0.9, 1.1))),  # brightness
            iaa.Sometimes(0.3, iaa.GaussianBlur(sigma=(0, 0.5))),
            iaa.Sometimes(0.2, iaa.AdditiveGaussianNoise(scale=(0, 0.02*255))),
        ])

    def extract_face(self,filename):
        img = cv.imread(filename)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        x,y,w,h = self.detector.detect_faces(img)[0]['box']
        x,y = abs(x),abs(y)
        face = img[y:y+h,x:x+w]
        face_arr= cv.resize(face,self.target_size)
        return face_arr
    def load_face(self, dir):
        FACES = []
        for im_name in os.listdir(dir):
            try:
                path = dir+im_name
                single_face= self.extract_face(path)
                FACES.append(single_face)

                for _ in range(3):
                    aug_face = self.augmenter(image = single_face)
                    FACES.append(aug_face)
            except Exception as e:
                print("face not detected")
                # pass
        return FACES
    
    def load_classes(self):
        for sub_dir in os.listdir(self.directory):
            path=self.directory + '/' + sub_dir + '/'
            FACES = self.load_face(path)
            labels = [sub_dir for _ in range(len(FACES))]
            print(f"Loaded successfully :{len(labels)}")
            self.X.extend(FACES)
            self.Y.extend(labels)
        return np.asarray(self.X), np.asarray(self.Y)


fl = faceloading(r"data")
X , Y = fl.load_classes()
# print(X)
# print(Y)

embedder = FaceNet()
def get_embeddings(face_img):
    face_img = face_img.astype("float32")
    face_img = np.expand_dims(face_img,axis = 0)
    yhat= embedder.embeddings(face_img)
    return yhat[0]


EMBEDDED_X = []
for img in X:
    EMBEDDED_X.append(get_embeddings(img))
EMBEDDED_X = np.asarray(EMBEDDED_X)

np.savez_compressed('embeddings/faces_embeddings_done_4classes.npz', EMBEDDED_X, Y)



# img = cv.imread(r"Me.jpg")
# img = cv.cvtColor(img,cv.COLOR_BGR2RGB)
# detector = MTCNN()
# result = detector.detect_faces(img)
# x,y,w,h = result[0]['box']
# img = cv.rectangle(img, (x,y), (x+w, y+h), (0,0,255), 30)

# my_face=  img[y:y+h, x:x+w]
# my_face = cv.resize(my_face,(160,160))
# plt.imshow(my_face)
# plt.show()


# print(img)
# img = cv.cvtColor(img,cv.COLOR_BGR2RGB)
# plt.imshow(img)
# plt.show()


In [2]:
# Model Training
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import pickle
import os


data = np.load('embeddings/faces_embeddings_done_4classes.npz')
X = data['arr_0']
Y = data['arr_1']

encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, shuffle=True, random_state=17)
model = SVC(kernel='linear', probability=True)
model.fit(X_train, Y_train)
ypreds_train = model.predict(X_train)
ypreds_test = model.predict(X_test)
print(accuracy_score(Y_train, ypreds_train))
print(accuracy_score(Y_test,ypreds_test))


os.makedirs('model', exist_ok=True)

with open('model/svm_model_160x160.pkl','wb') as f:
    pickle.dump(model,f)

with open('model/label_encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)


1.0
1.0


IndexError: tuple index out of range

In [ ]:
# Live cam

import cv2 as cv
from mtcnn.mtcnn import MTCNN
import pickle
# from generate_embeddings import get_embeddings
detector = MTCNN()
import numpy as np

from keras_facenet import FaceNet
embedder = FaceNet()
def get_embeddings(face_img):
    face_img = face_img.astype("float32")
    face_img = np.expand_dims(face_img,axis = 0)
    yhat= embedder.embeddings(face_img)
    return yhat[0]

with open('model/svm_model_160x160.pkl', 'rb') as f:
    model = pickle.load(f)

with open('model/label_encoder.pkl', 'rb') as g:
    encoder = pickle.load(g)
cap = cv.VideoCapture(0)
print("Press 'q' to quit.")
while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    faces = detector.detect_faces(rgb_frame)

    for face in faces:
        x, y, w, h = face['box']
        x, y = abs(x), abs(y)
        face_crop = rgb_frame[y:y+h, x:x+w]

        try:
            face_crop = cv.resize(face_crop, (160, 160))
            emb = get_embeddings(face_crop)

            # Predict probabilities for all classes
            if hasattr(model, "predict_proba"):
                proba = model.predict_proba([emb])[0]
                best_idx = np.argmax(proba)
                confidence = proba[best_idx]
                predicted_name = encoder.inverse_transform([best_idx])[0]

                # Set threshold for unknown
                threshold = 0.97
                # 91    
                # 98
                if confidence < threshold:
                    name = f"Unknown ({confidence:.2f})"
                else:
                    name = f"{predicted_name} ({confidence:.2f})"
            else:
                # Fallback if predict_proba is not available
                pred = model.predict([emb])
                name = encoder.inverse_transform(pred)[0]

            # Draw bounding box and label
            cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv.putText(frame, name, (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (36, 255, 12), 2)

        except Exception as e:
            print("Error in face processing:", e)

    cv.imshow('Real-time Face Recognition', frame)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


# Method 2

In [22]:
import cv2 as cv
import os
import numpy as np
from mtcnn import MTCNN

class Faceloading:
    def __init__(self, directory):
        self.directory = directory
        self.target_size = (160, 160)
        self.X = []
        self.Y = []
        self.detector = MTCNN()

    def extract_face(self, filename):
        img = cv.imread(filename)
        if img is None:  # Check if the image was loaded correctly
            print(f"Error loading image: {filename}")
            return None
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        faces = self.detector.detect_faces(img)
        if not faces:
            print(f"No faces detected in image: {filename}")
            return None
        x, y, w, h = faces[0]['box']
        x, y = abs(x), abs(y)
        face = img[y:y+h, x:x+w]
        face_arr = cv.resize(face, self.target_size)
        return face_arr

    def load_faces(self, dir):
        FACES = []
        for im_name in os.listdir(dir):
            try:
                path = os.path.join(dir, im_name)  # Correct path handling
                single_face = self.extract_face(path)
                if single_face is not None:  # Only append valid faces
                    FACES.append(single_face)
            except Exception as e:
                print(f"Error processing {im_name}: {e}")
        return FACES

    def load_classes(self):
        for sub_dir in os.listdir(self.directory):
            path = os.path.join(self.directory, sub_dir)  # Correct path handling
            if os.path.isdir(path):
                FACES = self.load_faces(path)
                labels = [sub_dir for _ in range(len(FACES))]
                print(f"Loaded successfully: {len(labels)}")
                self.X.extend(FACES)
                self.Y.extend(labels)
        return np.asarray(self.X), np.asarray(self.Y)

# Test the class

faceloading = Faceloading(r"data")
X, Y = faceloading.load_classes()

Loaded successfully: 100
Loaded successfully: 9
No faces detected in image: data\parn\41.jpg
No faces detected in image: data\parn\55.jpg
No faces detected in image: data\parn\65.jpg
No faces detected in image: data\parn\66.jpg
No faces detected in image: data\parn\80.jpg
No faces detected in image: data\parn\81.jpg
No faces detected in image: data\parn\82.jpg
No faces detected in image: data\parn\83.jpg
Loaded successfully: 92


In [23]:
X

array([[[[ 52,  66,  81],
         [ 57,  69,  81],
         [ 56,  67,  73],
         ...,
         [ 51,  62,  68],
         [ 63,  73,  82],
         [ 71,  84,  93]],

        [[ 52,  69,  83],
         [ 54,  68,  80],
         [ 53,  65,  74],
         ...,
         [ 52,  63,  72],
         [ 59,  70,  79],
         [ 77,  90,  99]],

        [[ 53,  71,  83],
         [ 50,  68,  80],
         [ 48,  64,  74],
         ...,
         [ 50,  63,  70],
         [ 60,  72,  80],
         [ 68,  81,  89]],

        ...,

        [[169, 150, 153],
         [169, 150, 152],
         [171, 153, 154],
         ...,
         [101,  83,  81],
         [104,  86,  84],
         [108,  88,  87]],

        [[168, 149, 150],
         [167, 148, 149],
         [169, 150, 151],
         ...,
         [103,  85,  83],
         [106,  88,  86],
         [108,  89,  89]],

        [[169, 151, 151],
         [169, 149, 150],
         [168, 148, 149],
         ...,
         [103,  85,  83],
        

In [24]:
# import os
# import torch
# import numpy as np
# from PIL import Image
# from facenet_pytorch import InceptionResnetV1
# from torchvision import transforms

# # Load the pre-trained FaceNet model
# model = InceptionResnetV1(pretrained='vggface2').eval()

# # Folder where face crops are saved
# FACE_DIR = "face_images"
# EMBEDDING_DIR = "embeddings"
# os.makedirs(EMBEDDING_DIR, exist_ok=True)

# # Preprocessing function
# preprocess = transforms.Compose([
#     transforms.Resize((160, 160)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.5], [0.5])
# ])

# def get_embedding(face_path):
#     img = Image.open(face_path).convert('RGB')
#     img_tensor = preprocess(img).unsqueeze(0)  # Add batch dim
#     with torch.no_grad():
#         embedding = model(img_tensor).numpy()
#     return embedding

# def process_all_faces():
#     for filename in os.listdir(FACE_DIR):
#         if filename.endswith(".jpg") or filename.endswith(".png"):
#             path = os.path.join(FACE_DIR, filename)
#             emb = get_embedding(path)
#             emb_path = os.path.join(EMBEDDING_DIR, filename.replace(".jpg", ".npy"))
#             np.save(emb_path, emb)
#             print(f"[✓] Saved embedding for {filename}")

# if __name__ == "__main__":
#     process_all_faces()





import os
import torch
import numpy as np
from PIL import Image
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms

# Load the pre-trained FaceNet model
model = InceptionResnetV1(pretrained='vggface2').eval()

# Folder where your dataset (person1, person2, ...) is saved
DATA_DIR = "data"
EMBEDDING_DIR = "embeddings"
os.makedirs(EMBEDDING_DIR, exist_ok=True)

# Preprocessing function
preprocess = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

def get_embedding(face_path):
    img = Image.open(face_path).convert('RGB')
    img_tensor = preprocess(img).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        embedding = model(img_tensor).numpy()
    return embedding

def process_all_faces():
    for person_name in os.listdir(DATA_DIR):  # Iterate through each person folder (person1, person2, ...)
        person_folder = os.path.join(DATA_DIR, person_name)
        if os.path.isdir(person_folder):
            person_embeddings = []  # List to store embeddings for this person
            for filename in os.listdir(person_folder):  # Iterate through each image in the person's folder
                if filename.endswith(".jpg") or filename.endswith(".png"):
                    path = os.path.join(person_folder, filename)
                    emb = get_embedding(path)
                    emb_path = os.path.join(EMBEDDING_DIR, f"{person_name}_{filename.replace('.jpg', '.npy').replace('.png', '.npy')}")
                    np.save(emb_path, emb)  # Save embedding as a .npy file
                    print(f"[✓] Saved embedding for {filename} of {person_name}")

if __name__ == "__main__":
    process_all_faces()


  0%|          | 0.00/107M [00:00<?, ?B/s]

[✓] Saved embedding for 0.jpg of Adivya
[✓] Saved embedding for 1.jpg of Adivya
[✓] Saved embedding for 10.jpg of Adivya
[✓] Saved embedding for 11.jpg of Adivya
[✓] Saved embedding for 12.jpg of Adivya
[✓] Saved embedding for 13.jpg of Adivya
[✓] Saved embedding for 14.jpg of Adivya
[✓] Saved embedding for 15.jpg of Adivya
[✓] Saved embedding for 16.jpg of Adivya
[✓] Saved embedding for 17.jpg of Adivya
[✓] Saved embedding for 18.jpg of Adivya
[✓] Saved embedding for 19.jpg of Adivya
[✓] Saved embedding for 2.jpg of Adivya
[✓] Saved embedding for 20.jpg of Adivya
[✓] Saved embedding for 21.jpg of Adivya
[✓] Saved embedding for 22.jpg of Adivya
[✓] Saved embedding for 23.jpg of Adivya
[✓] Saved embedding for 24.jpg of Adivya
[✓] Saved embedding for 25.jpg of Adivya
[✓] Saved embedding for 26.jpg of Adivya
[✓] Saved embedding for 27.jpg of Adivya
[✓] Saved embedding for 28.jpg of Adivya
[✓] Saved embedding for 29.jpg of Adivya
[✓] Saved embedding for 3.jpg of Adivya
[✓] Saved embedding 

[✓] Saved embedding for 99.jpg of parn


In [25]:
import os
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
import imgaug.augmenters as iaa
import faiss

class FaceExtractor:
    def __init__(self, dataset_path, target_size=(160, 160)):
        self.dataset_path = dataset_path
        self.target_size = target_size
        self.detector = MTCNN()
        self.augmenter = iaa.Sequential([
            iaa.Affine(rotate=(-10, 10)),
            iaa.Sometimes(0.3, iaa.Multiply((0.9, 1.1))),
            iaa.Sometimes(0.3, iaa.GaussianBlur(sigma=(0, 0.5))),
            iaa.Sometimes(0.2, iaa.AdditiveGaussianNoise(scale=(0, 0.02 * 255))),
        ])
        self.embedder = FaceNet()

    def extract_face(self, img_path):
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = self.detector.detect_faces(img_rgb)

        if not faces:
            return None

        x, y, w, h = faces[0]['box']
        x, y = abs(x), abs(y)
        face = img_rgb[y:y + h, x:x + w]
        face = cv2.resize(face, self.target_size)
        return face

    def generate_embeddings(self, faces):
        embeddings = []
        for face in faces:
            face = face.astype("float32")
            face = np.expand_dims(face, axis=0)
            embedding = self.embedder.embeddings(face)
            embeddings.append(embedding[0])  # Extract first (and only) embedding for face
        return np.asarray(embeddings)

    def load_dataset(self):
        faces = []
        labels = []

        for person_name in os.listdir(self.dataset_path):
            person_dir = os.path.join(self.dataset_path, person_name)
            if not os.path.isdir(person_dir):
                continue

            print(f"[+] Processing {person_name}...")
            for img_name in os.listdir(person_dir):
                img_path = os.path.join(person_dir, img_name)
                try:
                    face = self.extract_face(img_path)
                    if face is not None:
                        faces.append(face)
                        labels.append(person_name)

                        # Augment face ×3
                        for _ in range(3):
                            aug_face = self.augmenter(image=face)
                            faces.append(aug_face)
                            labels.append(person_name)
                except Exception as e:
                    print(f"[-] Error with {img_path}: {e}")
                    continue

        print(f"\n✅ Total faces: {len(faces)}")
        return np.asarray(faces), np.asarray(labels)

    def save_embeddings(self, faces, labels):
        embeddings = self.generate_embeddings(faces)
        # Save embeddings and labels as npz
        np.savez_compressed("faces_embeddings_with_labels.npz", embeddings=embeddings, labels=labels)
        print("✅ Saved embeddings and labels to 'faces_embeddings_with_labels.npz'")

# ---- FAISS Index Code ----
def build_faiss_index(embeddings):
    # Dimension of the embeddings (128 for FaceNet)
    d = embeddings.shape[1]  # Assuming FaceNet embeddings of size (128,)
    print(f"[✓] Using embeddings of dimension: {d}")

    # Create a FAISS index (using L2 distance metric)
    index = faiss.IndexFlatL2(d)
    
    # Add embeddings to the index
    index.add(embeddings)
    
    print(f"[✓] FAISS index built with {embeddings.shape[0]} faces")
    return index

# ---- Run the full script ----
if __name__ == "__main__":
    dataset_dir = "data"  # Your folder with 3 subfolders
    extractor = FaceExtractor(dataset_dir)
    faces, labels = extractor.load_dataset()

    print(f"✅ Loaded faces: {faces.shape}")
    print(f"✅ Loaded labels: {labels.shape}")

    # Save embeddings and labels
    extractor.save_embeddings(faces, labels)

    # Load saved embeddings for FAISS usage
    data = np.load("faces_embeddings_with_labels.npz")
    embeddings = data['embeddings']
    labels = data['labels']
    
    # Build FAISS index from embeddings
    faiss_index = build_faiss_index(embeddings)

    # Now you can perform similarity search with FAISS on the index
    # Example: search for the top 5 closest matches for a given query embedding
    query_embedding = embeddings[0]  # Use the first face as a query
    k = 5  # Number of nearest neighbors to retrieve
    D, I = faiss_index.search(np.expand_dims(query_embedding, axis=0), k)  # D: distances, I: indices
    print(f"Closest matches: {I[0]} with distances: {D[0]}")



[+] Processing Adivya...
[+] Processing bill...
[+] Processing parn...

✅ Total faces: 804
✅ Loaded faces: (804, 160, 160, 3)
✅ Loaded labels: (804,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━

In [27]:
import cv2
import numpy as np
import faiss
from keras_facenet import FaceNet
from mtcnn.mtcnn import MTCNN

# Load the pre-built FAISS index and embeddings
data = np.load("faces_embeddings_with_labels.npz")
embeddings = data['embeddings']
labels = data['labels']

# Build the FAISS index
def build_faiss_index(embeddings):
    d = embeddings.shape[1]  # Dimensions of the embeddings
    index = faiss.IndexFlatL2(d)  # L2 distance metric
    index.add(embeddings)  # Add embeddings to the index
    return index

faiss_index = build_faiss_index(embeddings)

# Initialize FaceNet and MTCNN
face_net = FaceNet()
detector = MTCNN()

# Define threshold for "unknown" face detection
threshold = 1.0  # You can adjust this threshold to be stricter or looser

# Start the webcam feed
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces in the frame
    faces = detector.detect_faces(frame)
    
    for face in faces:
        x, y, w, h = face['box']
        face_img = frame[y:y+h, x:x+w]
        
        # Resize face image to 160x160 for FaceNet
        face_img_resized = cv2.resize(face_img, (160, 160))
        
        # Get embedding for the detected face
        face_embedding = face_net.embeddings(np.expand_dims(face_img_resized, axis=0))[0]
        
        # Use FAISS to find the closest match
        k = 1  # Only retrieve the closest match
        D, I = faiss_index.search(np.expand_dims(face_embedding, axis=0), k)
        
        # Get the closest match distance and label
        distance = D[0][0]
        label = labels[I[0][0]]
        
        # Check if the distance is above the threshold
        if distance > threshold:
            label = "Unknown"  # Label as unknown if the distance is too large
        else:
            label = f"{label} ({distance:.2f})"  # Display the match with distance

        # Draw bounding box and label on the frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Live Face Recognition', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━